In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from torch.nn import Sequential
from torch.nn import Linear, Conv2d, MaxPool2d
from torch.nn import Sigmoid, Softmax, ReLU

from torch.optim import SGD

from torch.utils.data import DataLoader, TensorDataset

from torch.nn import CrossEntropyLoss

In [4]:
Big_Table=pd.read_csv('./LA1_5lable.csv',index_col=[0],encoding='utf-8',low_memory=False)
Three_Day_Predict=pd.read_csv('./LA1_int.csv',index_col=[0],encoding='utf-8',low_memory=False)

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
Big_Table['ObservDay_time']=pd.to_datetime(Big_Table['ObservDay_time'])

In [6]:
Big_Table['DayofWeek']=Big_Table['ObservDay_time'].dt.dayofweek

In [7]:
Big_Table_ordered=Big_Table[['StationCode', 'StationNmae', 'ObservDay_time', 'month', 'DayofWeek','StnPres',
       'Temperature', 'RH', 'RHMin', 'WS', 'WD', 'WSGust', 'WDGust', 'Precp',
       'ty_generate', 'sea_warning', 'land_warning', 'ty_strength', 'ty_path',
       'festival_count', 'festival', 'altitude', 'altitude_classification',
       'locate_classification', 'Market', 'MediumAvgPrice', 'TradingVolume',
       'T+1', 'T+3', 'T+5', 'T+7', 'T+1_label', 'T+3_label', 'T+5_label',
       'T+7_label', ]]
Big_Table_ordered

,StationCode,StationNmae,ObservDay_time,month,DayofWeek,StnPres,Temperature,RH,RHMin,WS,...,MediumAvgPrice,TradingVolume,T+1,T+3,T+5,T+7,T+1_label,T+3_label,T+5_label,T+7_label
locate,,,,,,,,,,,,,,,,,,,,,
Changhua,467270,田中,2020-04-09,4,3,1011.100000,22.100,77.000000,41.000000,2.10,...,9.5,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0
Changhua,C0G620,芬園,2020-04-09,4,3,1003.200000,22.000,75.000000,32.000000,1.40,...,9.5,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0
Changhua,C0G640,鹿港,2020-04-09,4,3,1017.300000,21.200,73.000000,54.000000,2.40,...,9.5,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0
Changhua,C0G650,員林,2020-04-09,4,3,1012.200000,22.500,79.000000,37.000000,0.80,...,9.5,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0
Changhua,C0G660,溪湖,2020-04-09,4,3,1014.800000,22.100,71.000000,41.000000,2.50,...,9.5,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yilan,C1U670,寒溪,2011-10-30,10,6,1004.350000,21.825,74.500000,66.500000,1.75,...,13.1,135788.0,0.130,-0.233,-0.454,-0.260,4.0,2.0,1.0,1.0
Yilan,C1U690,新寮,2011-10-30,10,6,1004.350000,21.825,74.500000,66.500000,1.75,...,13.1,135788.0,0.130,-0.233,-0.454,-0.260,4.0,2.0,1.0,1.0
Yunlin,C0K280,四湖,2011-10-30,10,6,1002.310135,23.500,72.868998,61.536329,3.20,...,13.1,135788.0,0.130,-0.233,-0.454,-0.260,4.0,2.0,1.0,1.0


In [8]:
Big_Table_ordered_drop=Big_Table_ordered.drop(columns=['StationCode', 'StationNmae', 'ObservDay_time','RHMin','WSGust','festival','altitude', 'altitude_classification','locate_classification', 'Market', 'MediumAvgPrice',
       'T+1', 'T+3', 'T+5', 'T+7', 'T+3_label','T+1_label','T+5_label','TradingVolume','land_warning','ty_strength','ty_generate'])

In [9]:
# Big_Table_ordered_drop['month']=Big_Table_ordered_drop['month'].astype(float)
# Big_Table_ordered_drop['DayofWeek']=Big_Table_ordered_drop['DayofWeek'].astype(float)
# Big_Table_ordered_drop['sea_warning']=Big_Table_ordered_drop['sea_warning'].astype(float)
# Big_Table_ordered_drop['ty_path']=Big_Table_ordered_drop['ty_path'].astype(float)
# Big_Table_ordered_drop['festival_count']=Big_Table_ordered_drop['festival_count'].astype(float)

In [10]:
Big_Table_ordered_drop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1356259 entries, Changhua to Yunlin
Data columns (total 13 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   month           1356259 non-null  int64  
 1   DayofWeek       1356259 non-null  int64  
 2   StnPres         1356259 non-null  float64
 3   Temperature     1356259 non-null  float64
 4   RH              1356259 non-null  float64
 5   WS              1356259 non-null  float64
 6   WD              1356259 non-null  float64
 7   WDGust          1356259 non-null  float64
 8   Precp           1356259 non-null  float64
 9   sea_warning     1356259 non-null  int64  
 10  ty_path         1356259 non-null  int64  
 11  festival_count  1356259 non-null  int64  
 12  T+7_label       1356259 non-null  float64
dtypes: float64(8), int64(5)
memory usage: 144.9+ MB


In [12]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(Big_Table_ordered_drop):
    device = get_device()
    return torch.from_numpy(Big_Table_ordered_drop.values).float().to(device)

df_tensor = df_to_tensor(Big_Table_ordered_drop)
series_tensor = df_to_tensor(Big_Table_ordered_drop.iloc[:,0:12])

In [13]:
# 切分資料
X =series_tensor
y = Big_Table_ordered_drop.iloc[:,12].astype(float)
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

---

# 練習: 模型建立

In [14]:
from torch.nn import ReLU,Linear,Softmax
from torch.nn import Sequential

In [15]:
in_features = 12
out_classes = 5

act = Softmax()
# act = ReLU()

layers = [Linear(in_features, 5),
          act,
          Linear(5, 7),
          act,
          Linear(7, out_classes),
          Softmax(),]

In [16]:
model=Sequential(*layers)
model

Sequential(
  (0): Linear(in_features=12, out_features=5, bias=True)
  (1): Softmax(dim=None)
  (2): Linear(in_features=5, out_features=7, bias=True)
  (3): Softmax(dim=None)
  (4): Linear(in_features=7, out_features=5, bias=True)
  (5): Softmax(dim=None)
)

# 練習: 模型訓練

In [17]:
def train_step(dataloader, model, loss_fn, optimizer, verbose_every=999):
    """訓練一個epoch。
    """
    for iteration, (batch_x, batch_y) in enumerate(dataloader):

        optimizer.zero_grad()

        pred_y = model(batch_x)

        loss_value = ce_loss(pred_y, batch_y)
        loss_value.backward()

        if iteration+1 % verbose_every == 0:
            loss = loss_value.item()
            print("loss={:.4f}".format(iteration, loss))

        optimizer.step()


def test_step(dataloader, model, loss_fn):
    """結束一個epoch的訓練後，測試模型表現。
    """
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for iteration, (batch_x, batch_y) in enumerate(dataloader):
            pred_y = model(batch_x)

            test_loss += loss_fn(pred_y, batch_y).item()
            correct += (pred_y.argmax(axis=1) == batch_y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    
    print("test_loss={:.4f}, accuracy={:.2f}".format(test_loss, correct))


In [18]:
# 載入資料成為DataLoader
y = torch.tensor(y).to(torch.int64)
td = TensorDataset(X, y)
dl = DataLoader(td, batch_size=10, shuffle=True)

# 練習: 模型預測

In [ ]:
# 宣告模型訓練設定
num_epochs = 20
learning_rate = 0.1

# 定義優化器, Loss函數
ce_loss = CrossEntropyLoss()
opt = SGD(model.parameters(), lr=learning_rate)

# 訓練模型
for j in range(num_epochs):
    train_step(dl, model, ce_loss, opt)
    test_step(dl, model, ce_loss)

---